In [ ]:
import requests as rqs
import time
from bs4 import BeautifulSoup as soup
import threading 
cookies={}

После некоторого раздумия, сервис **[hidemy.name](https://hidemy.name/ru/proxy-list/)** показался мне предпочтительнее **[proxyrotator.com](https://www.proxyrotator.com/free-proxy-list/)**. Если на втором можно только получить список прокси, а потом отбирать нужные, то на первом можно задавать множество параметров поиска. Что приятно. Если уж быть абсолютно честным до конца, именно первый я по началу и хотел использовать. Но испугала процедура проверки веб-клиента на "человечность". Фильтрация невидимых элементов и распознавание чисел на картинках (**[proxyrotator.com](https://www.proxyrotator.com/free-proxy-list/)**) показалось занятием менее сложным.   

Если Вы удалите куки, связанные с **hidemy.name** и перейдёте по соответствующей **[ссылке](https://hidemy.name/ru/proxy-list/)**, сначала появится страничка, проверяющая браузер на "человечность" и только потом загрузится список прокси.  

Давайте попробуем прочитать страничку с **[hidemy.name](https://hidemy.name/ru/proxy-list/)** наиболее тупым способом, самой низкоуровневой питоньей библиотекой **urllib**

In [ ]:
import urllib.request as urlrq
import urllib.error as err

url="https://hidemy.name/ru/proxy-list/"
try:
    urlrq.urlopen(url)
except err.URLError as e :
    print(e.code)
    s=str(e.read().decode("utf-8"))
    file=open("hideme_403.html", "wt")
    file.write(s)
    file.close()

#Попробуйте здесь закомментировать или наоборот раскомментировать некоторые заголовки    
rq=urlrq.Request(url)
rq.add_header("User-Agent", "Mozilla/5.0 (X11; Linux x86_64; rv:59.0) Gecko/20100101 Firefox/59.0")
rq.add_header("Accept", "text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8")
#rq.add_header("Accept-Encoding", "gzip, deflate")
#rq.add_header("Accept-Language", "ru-RU,ru;q=0.8,en-US;q=0.5,en;q=0.3")
#rq.add_header("Connection", "keep-alive")
try:
    urlrq.urlopen(rq)
except err.URLError as e :
    print(e.code)
    s=str(e.read().decode("utf-8"))
    file=open("hideme_503.html", "wt")
    file.write(s)
    file.close()    

В первом случае получили 403. Откройте сохранённый файл, и Вы не увидите ничего похожего на проверочную страничку !    

Во втором (если Вы конечно не закомментировали необходимые заголовки), получаем 503, а сохранённый файл выглядит так, как он и должен выглядеть. 

Этот пример чисто учебный. На самом деле мы будем пользоваться более высокоуровневой библиотекой **requests**, которая судя по всему добавляет какие-то более не менее вменяемые заголовки по умолчанию (не проверял этот вопрос) и всегда получает 503. Но наш первый пример показывает, насколько важны тут заголовки. И намекает на то, что неплохо бы не надеяться на умолчательные, а собирать их ручками. Давайте так и сделаем. 

In [ ]:
#Это я просто выдрал из своей огненной лисички
firefox={
"Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8",    
"Accept-Encoding": "gzip, deflate", 
"Accept-Language": "ru-RU,ru;q=0.8,en-US;q=0.5,en;q=0.3",
"Connection": "keep-alive",
"Host": "hidemy.name",
"Referer": "https://hidemy.name/en/",        
"Upgrade-Insecure-Requests": "1",    
"User-Agent":"Mozilla/5.0 (X11; Linux x86_64; rv:59.0) Gecko/20100101 Firefox/59.0"   
}

def hidemy():
    return rqs.get("https://hidemy.name/en/proxy-list/", headers=firefox)

rsp=hidemy()
s=str(rsp.content.decode(rsp.encoding))
file=open("hideme_start.html", "wt")
file.write(s)
file.close()
bsObj=soup(s, "lxml")
print(rsp.status_code)
rsp.headers

Получили 503. В заголовке пришла кука с именем **__cfduid**, а сохранённый файл содержит немудрящий тест. Нужно подождать четыре секунды, потом выполнить несложный арифметический пример, забить его в форму и отправить эту форму на сервер. Всё бы ничего, но числа для этого примера записаны немного странным образом. Например вот это "число" - **+((!+[]+!![]+!![]+!![]+[])+(!+[]+!![]+!![]))** на самом деле равно **43**. Спасибо **j0hnik**, который помог с этим разобраться. Прежде всего давайте соберём все эти "числа". Посмотрев на файл видим, что сначала "число" присваивается некоторой переменной. А затем эта переменная участвует в операциях **+=**, **-=** и ***=**.  
В строке где переменной присваивается значение, много запятых, что позволяет её легко найти. Сделаем это: 

In [ ]:
def findNums(bsObj):
    res=[]                                         #результат
    script=str(bsObj.find("script").contents[0])   #находим javascript
    a=script.split("\n")                           #разбиваем по символам новой строки
    mx=0                                 #максимум числа запятых в строке
    idx=0                                #местоположение максимума
    for i in range(0, len(a)):      #ищем строку с максимальным количеством запятых
        aa=a[i].split(",")
        if len(aa)>mx:
            mx=len(aa)
            idx=i
    s=a[idx]                        #берём её часть справа от последней запятой
    s=s[s.rfind(",")+1:].strip()    #и удаляем концевые пробелы
    aa=s.split("=")                 #разбиваем по знаку =
    var=aa[0]                       #слева имя словаря
    aa=s.split('"')                 #разбиваем по знаку '"'
    var=var+"."+aa[1]               #добавляем имя поля
    aa=s.split(":")                 #разбиваем по знаку :
    res.append(aa[1][:aa[1].find("}")].strip()) #сохраняем то что справа од : до }
    aa=script.split(var)            #разбиваем весь скрипт по имени переменной
    for i in range(1, len(aa)-1):   #для всех подстрок от второй до предпоследней
        res.append(aa[i][:aa[i].find(";")]) #сохраняем от начала до ;
    return res

In [ ]:
#Проверяем как это работает
rsp=hidemy()
s=str(rsp.content.decode(rsp.encoding))
bsObj=soup(s, "lxml")
nums=findNums(bsObj)
print("Получено "+str(len(nums))+' "чисел"')
for n in nums:
    print(n)
# Запустите этот тест несколько раз, чтобы посмотреть какие "числа" вываливаются 
# У меня получалось от 2 до 12 "чисел"

Вычислим "число". Метод основан на моей беседе с **j0hnik** и сильно упрощён. Тем не менее те "числа" которые реально приходят, практически всегда считает правильно. Хотя 100% гарантии я дать не могу. Но 100% и не нужно. Если ошибёмся, попытку всегда можно повторить.

In [ ]:
def evalNun(s):
    if s[0]!="+":        #если "число" начинается не с + оно не содержит 
        a=s.split("[]")  #групп в круглых скобках. Просто считаем число подстрок "[]"
        return len(a)-1
    else:
        ss=""
        a=s.split(")")              # разбиваем на группы в круглых скобках
        for i in range(0, len(a)):
            if a[i]!="":
                s=a[i][a[i].rfind("(")+1:]  #удаляем открывающиеся скобки
                aa=s.split("[]")            #разбиваем по подстрокам []
                n=0
                for q in aa:       #если длинна предшествующей строки больше 1
                    if len(q)>1:   #то это 1
                        n+=1
                ss=ss+str(n) #интерпретируем полученное число, как символ числовой строки  
        return int(ss)        

In [ ]:
# Проверим как это работает
rsp=hidemy()
s=str(rsp.content.decode(rsp.encoding))
bsObj=soup(s, "lxml")
nums=findNums(bsObj)
print("Получено "+str(len(nums))+' "чисел"')
print(evalNun(nums[0]))
for i in range(1, len(nums)):
    a=nums[i].split("=")
    print(evalNun(a[1]))
print("====================================")
print("console.log("+nums[0]+")")
for i in range(1, len(nums)):
    a=nums[i].split("=")
    print("console.log("+a[1]+")")

# То что распечатано после ============ скопируйте и вставьте в консоль браузера. 
# Убедитесь, что список чисел вверху совпадает с тем, что будет вычислено в консоли. 
# Сделайте этот тест несколько раз, чтобы убедиться, что если алгоритм и ошибается,
# то весьма и весьма редко.

Теперь можем вычислить весь наш пример. "числа" мы сохраняли вместе с арифметическими операциями слева от знака =. Если посмотреть сохранённый файл, видно что в конце примера к результату надо прибавить **t.length**. Кэп подсказывает, что **t** это ни что иное, как строка **hidemy.name**. Делаем.

In [ ]:
# Код совершенно очевиден
def evalAnsw(a):
    num=evalNun(a[0])
    for i in range(1, len(a)):
        f=False
        aa=a[i].split("=")
        nn=evalNun(aa[1])
        if aa[0]=="+":
            f=True
            num+=nn
        if aa[0]=="-":
            f=True
            num-=nn
        if aa[0]=="*":
            f=True
            num*=nn
        if f==False:
            print("Unknown "+aa[0])
    return num+len("hidemy.name")  

Настало время собрать всё до кучи. И результат примера и форму и куку.

In [ ]:
# Собирает до кучи все параметры
# Возвращает тройку url, cookies, param
def getParams(rsp):
    s=str(rsp.content.decode(rsp.encoding))
    bsObj=soup(s, "lxml")
    a=findNums(bsObj)
    answ=evalAnsw(a)                               #решаем пример
    form=bsObj.find("form")                        #находим форму
    url="https://hidemy.name"+form.attrs["action"] #добавляем её action к url
    inps=form.find_all("input")                    #находим все элементы input
    params={}
    for i in range(0, len(inps)):
        val=inps[i].get("value")                   #если value нет
        if val==None:                              #подставляем туда пример
            val=answ
        params[inps[i].attrs["name"]]=val           #собираем до кучи
    cookeis=rsp.cookies.get_dict()
    return (url, cookeis, params)

getParams(rsp)

И вот он, критический момент !

In [ ]:
#Перепишем firefox поближе, чтобы было удобно экспериментировать
firefox={
"Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8",    
"Accept-Encoding": "gzip, deflate", 
"Accept-Language": "ru-RU,ru;q=0.8,en-US;q=0.5,en;q=0.3",
"Connection": "keep-alive",
"Host": "hidemy.name",
"Referer": "https://hidemy.name/en/proxy-list/",    
"Upgrade-Insecure-Requests": "1",    
"User-Agent":"Mozilla/5.0 (X11; Linux x86_64; rv:59.0) Gecko/20100101 Firefox/59.0"   
}

rsp=hidemy()
url, cookies, params=getParams(rsp)
time.sleep(4)
rsp1=rqs.get(url, headers=firefox, cookies=cookies, params=params)
s=str(rsp1.content.decode(rsp1.encoding))
print(rsp1.status_code)
bsObj=soup(s, "lxml")
title=bsObj.find("title")
print(str(title.contents[0]))
file=open("hideme_proxy.html", "wt")
file.write(s)
file.close()

Получили 200. Открываем сохраненный файл **hideme_proxy.html** и убеждаемся что пришел список прокси. Попробуем чуть-чуть "пошевелить" полученный результат. например поставить время ожидания 2 секунды или 30 секунд вместо 4. Или прибавить 1 к результату примера. Или изменить в заголовке поле Referer. Всё сразу перестаёт работать. Для гурманов-садомазо - попробуйте в поле заголовка **Accept-Encoding** добавить **br** (чтобы было **"Accept-Encoding": "gzip, deflate, br", **), получится смешно. Смех смехом, но между прочим firefox именно такое значение этого поля и прописывает. На результат я ффтыкал часа наверно три с гаком, не понимая кто меня дурит. Потом уже **br** от туда убрал. 

Попробуем, можно ли получать прокси с установленной кукой, чтобы не ждать 4 секунды и не решать дурацких примеров по арифметике.

In [ ]:
url="https://hidemy.name/ru/proxy-list/"
test=rqs.get(url, headers=firefox, cookies=cookies)
s=str(test.content.decode(test.encoding))
print(test.status_code)
bsObj=soup(s, "lxml")
title=bsObj.find("title")
print(str(title.contents[0]))
file=open("hideme_test.html", "wt")
file.write(s)
file.close()

Не работает... Дело в том, что **rsp1** не простой ответ. На самом деле сервер сначала отвечает кодом **302**, означающим редирект. А уже после происходит сам редирект, с кодом **200**. Высокоуровневая библиотека **requests** этот момент скрывает. Но вот незадача, в ответе с кодом **302** приходит ещё одна кука. Естественно она у нас потеряна. Но она лежит в истории **rsp1**, откуда её можно достать. Сделаем это:

In [ ]:
rsp302=rsp1.history[0]
print(rsp302.status_code)
cookies302=rsp302.cookies.get_dict()
print(cookies302)
cookies.update(cookies302)
print(cookies)

Попробуем снова тот же самый тест

In [ ]:
url="https://hidemy.name/en/proxy-list/"
test=rqs.get(url, headers=firefox, cookies=cookies)
s=str(test.content.decode(test.encoding))
print(test.status_code)
bsObj=soup(s, "lxml")
title=bsObj.find("title")
print(str(title.contents[0]))
file=open("hideme_test.html", "wt")
file.write(s)
file.close()

Всё работает ! Отлично ! Теперь можно написать функцию, которая будет обращаться к сайту и при необходимости обновлять куки (они выдаются на сутки). Наша функция будет собирать прокси с указанного числа страничек (на каждой страничке 64 прокси), задавая их поиск параметрами поиска на самом сайте. Эти параметры следующие:
1) Анонимность (высокая, средняя, низкая, нет)    
2) Протокол (http, https, sock4, sock5)    
3) Скорость не менее заданной (мс)    
4) Страна    
5) Порт    
Чтобы не таскать всё это с собой, зададим параметрам значения по умолчанию:    
Анонимность - высокая (5)        
Протокол    - http (h)  
Скорость    - 2000    
Страна      - все     
Порт        - все    
Такие значения параметров удобны мне. У Вас могут быть другие предпочтения.

In [ ]:
cookies={'__cfduid': 'dd1b26e2267ead10f15363f4066053f681517022909',
 'cf_clearance': '511ce2492b2ae5a2b2cfeb2973de73ff2921fd63-1517022914-86400'}

# Делает запрос с заданными параметрами
# Если не получает ответа 200, пытается обновить куки
# и повторить тот же запрос
def reCookies(params):
    global cookies
    firefox={
    "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8",    
    "Accept-Encoding": "gzip, deflate", 
    "Accept-Language": "ru-RU,ru;q=0.8,en-US;q=0.5,en;q=0.3",
    "Connection": "keep-alive",
    "Host": "hidemy.name",
    "Referer": "https://hidemy.name/en/proxy-list/",    
    "Upgrade-Insecure-Requests": "1",    
    "User-Agent":"Mozilla/5.0 (X11; Linux x86_64; rv:59.0) Gecko/20100101 Firefox/59.0"   
    }
    url="https://hidemy.name/en/proxy-list/"  
    if params!="":
        url=url+"?"+params
    url=url+"#list"
    for i in range(0, 10): #делаем 10 попыток
        try:
            rsp=rqs.get(url, headers=firefox, cookies=cookies)
        except:
            return None
        if rsp.status_code==200:
            return rsp
        else:
            url1, cookies, param1=getParams(rsp)  #получаем параметры обновления
            time.sleep(4)    #ждем 4 секунды и обновляемся
            try:
                rsp=rqs.get(url1, headers=firefox, params=param1, cookies=cookies)
            except:
                return None
            if rsp.status_code==200: #если получили 200, обновляем вторую куку
                rsp302=rsp.history[0]
                cookies302=rsp302.cookies.get_dict()
                cookies.update(cookies302)      
    return None #попытки закончились            
   
#https://hidemy.name/ru/proxy-list/?country=CAFRDEUS&maxtime=2000&type=h&anon=234&start=64#list
#https://hidemy.name/ru/proxy-list/?country=CAFRDEUS&maxtime=2000&ports=8080&type=h&anon=234#list
def scanHidemyPage(page, anon="4", protocol="h", speed=2000, country="", ports=""):
    res=[]  #результат
    #собираем параметры
    #Увы, стандартно в виде словаря их передавать нельзя ибо на сайте 
    #проверяется порядок их следования. Поэтому их нужно явно собирать
    #и передавать как часть строки запроса. 
    params=""
    if country!="":
        params=params+"&country="+country
    if speed!=0:
        params=params+"&maxtime="+str(speed)
    if ports!="":
        params=params+"&ports="+ports
    if protocol!="":
        params=params+"&type="+protocol
    if anon!="":
        params=params+"&anon="+anon
    if page>1:
        params=params+"&start="+str((page-1)*64)
        
    params=params[1:]    
    npages=-1              #число страниц в списке прокси
    rsp=reCookies(params)  #получаем страницу сайта
    if rsp==None:
        return (npages, res)
    try:
        s=str(rsp.content.decode(rsp.encoding))
        bsObj=soup(s, "lxml")
        #Первым делом смотрим, сколько вообще есть страничек
        pages=None
        divs=bsObj.find_all("div") #ищем все элементы div
        for div in divs:
            cc=div.attrs.get("class")
            if cc!=None:
                if cc[0]=="proxy__pagination":
                    pages=div                 #выбираем div с классом proxy__pagination
                    break
        if pages==None:
            return (-1, res)
        aa=pages.find_all("a")                # ищем все теги a
        #в тегах а ищем наибольшее число в содержимом
        for a in aa:
            ct=a.contents
            if len(ct)>0:
                nn=int(ct[0])
                if nn>npages:
                    npages=nn
        if npages<page:                        #если страница больше чем число страниц
            return (npages, res)               #читать разумеется нечего
        #читаем прокси
        trs=bsObj.find("tbody").find_all("tr") #получаем строки таблицы прокси
        #Читаем из таблицы. Каждое поле имеет свою специфику, и код не комментирую 
        for tr in trs:
            proxy={}                           #создаем новый прокси
            tds=tr.find_all("td")              #получаем td и читаем из них
            proxy["ip"]=str(tds[0].contents[0]).strip()
            proxy["port"]=str(tds[1].contents[0]).strip()
            div=tds[2].contents[0]
            proxy["country"]=str(div.contents[1]).strip()
            proxy["city"]=""
            if len(div.contents)>2:
                proxy["city"]+"|"+str(div.contents[2]).strip()
            p=tds[3].find("p")
            proxy["speed"]=int(p.contents[0].split(" ")[0])
            pt=tds[4].contents[0].split(",")
            for i in range(0, len(pt)):
                pt[i]=pt[i].strip()
            #print("speed")    
            proxy["protocol"]=pt
            anons=["No","Low", "Medium", "High"]
            an=tds[5].contents[0].strip()
            if an in anons:
                proxy["anon"]=anons.index(an)
            else:
                proxy["anon"]=-1
            ago=tds[6].contents[0].split(" ")
            proxy["ago"]=int(ago[0])*60
            if len(ago)>3:
                proxy["ago"]=60*(60*int(ago[0])+int(ago[2]))
            res.append(proxy)        
    except:
        return (npages, res)#произошла какая-то неожиданность
    return (npages, res)


In [ ]:
#Проверяем
np, proxies=scanHidemyPage(1)
print(np)
print(proxies)

Проверим теперь как наши прокси работают. Как и с **[proxyrotator](https://www.proxyrotator.com/free-proxy-list/)**, используем для этого ресурс **[icanhazip](http://icanhazip.com/)**.   
Внимание !!! Тест будет выполняться несколько минут !

In [ ]:
import math
urlIpTest="http://icanhazip.com/"
s=""
try:
    rsp=rqs.get(urlIpTest)
    s=str(rsp.content.decode(rsp.encoding))
except:
    s="Ошибка"
print("Наш ip-адрес  ---> "+s)

timeout=5    #Зададим таймаут 5 секунд, чтобы не ждать слишком медленных прокси
             #Вы эксперимента ради, можете задать его и больше. Меньше уже нежелательно
             #Слишком много серверов отвалятся с ошибкой
n=0
nn=0
speed=0     #Здесь Вы можете задать отбор прокси по скорости, от 1 до 5  
for proxy in proxies:
    n+=1
    server=proxy["ip"]+":"+str(proxy["port"])
    prox={"http" : server}
    t1=time.time()
    try:
        rsp=rqs.get(urlIpTest, proxies=prox, timeout=timeout)
        s=str(rsp.content.decode(rsp.encoding))
        if(s[0]=="<"):
            s="Забанен"
        else:
            nn+=1
    except:
        s="Ошибка"
    t2=time.time()    
    t=str(math.floor(t2-t1))
    print(server+" v="+str(proxy["speed"])+" => "+s+" t="+t)
print("Выбрано прокси: "+str(n))
print("Без ошибок: "+str(nn))


Видно что ошибок сыплется много и прокси нужно проверять. Так же как и раньше, напишем функцию, пытающуюся найти не менее заданного количества прокси, работающих с нужным сайтом. Но поскольку теперь я пишу код не для себя, а предполагаю что им могут воспользоваться другие, я попытался сделать её более универсальной и полезной. Функция будет получать на вход проверочный url и функцию отбора. Функция отбора получает на вход объект bsObj и прокси с которого он был получен. Возвращать она должна **True** если прокси принят и **False** если отвергнут. Так же для связи с проверочным сайтом можно задавать заголовки, куки и параметры.

In [ ]:
#Получает список прокси с hidemy.name
#Проверяет каждый попыткой соединения c url
#проверяя результат функцией отбора selFunc
def getHidemyProxyList(nProxy, url, selFunc, hdrs={}, cooks={}, pars={}, tout=5, 
                anon="4", protocol="h", speed=2000, country="", ports=""):
    nAccept=0                    #число принятых прокси
    nth=0                        #число потоков
    lock=threading.Lock()        #замок
    done=threading.Event()       #событие окончания всех проверок

    #Вложенная функция пытающаяся соединиться проверочным сервером
    #и в случае получения какого-то ответа проверить, а тот ли это ответ что нужен
    def test(p):                
        nonlocal url, selFunc, nAccept, nth, lock, done 
        f=False
        proxy={"http":p["ip"]+":"+str(p["port"])} 
        #Соединяемся и проверяем содержимое странички
        #В случае успеха устанавливаем флаг f в True
        try: 
            rsp=rqs.get(url, proxies=proxy, cookies=cooks, params=pars, timeout=tout)
            s=str(rsp.content.decode(rsp.encoding))
            bsObj=soup(s, "lxml") 
            if selFunc(p, bsObj):
                f=True
        except:
            f=False
        lock.acquire()     #Запираем замок, тут должно быть только однопоточное исполнение
        if nth>0:          #Уменьшаем число потоков
            nth-=1
        if f:              #Если проверили успешно, увеличиваем число принятых прокси
            nAccept+=1
        if nth==0:         #Если число потоков равно нулю, запускаем событие
            done.set()
        lock.release()     #освобождаем замок
    
    page=1                 #страничка hidemy.name
    npages=10              #число страничек
    while page<=npages:    #пока есть доступные странички
        #получаем страничку
        npages, proxy=scanHidemyPage(page, anon=anon, protocol=protocol, 
                                   speed=speed, country=country, ports=ports)
        if len(proxy)==0:
            return page  #значит нечего больше делать
        for p in proxy:  #для каждого прокси запускаем проверку в отдельном потоке
            nth+=1     
            th=threading.Thread(target=test, args=[p])
            th.start()
        done.wait()      #ждем окончания проверок
        if nAccept>=nProxy:
            return page  #отваливаемся, если собрали достаточно 
        page+=1          #переходим к следующей страничке
    return page

Поскольку я работаю с сайтом избиркома, на нём и проверим.

In [ ]:
izbircomURL="http://www.vybory.izbirkom.ru/region/izbirkom?action=show&global=1&vrn=100100031793505&region=0&prver=0&pronetvd=null"

result=[]  #сюда будем складывать отобранные прокси
def izbircomTest(proxy, bsObj):
    try:
        prx=proxy["ip"]+":"+proxy["port"]
        if prx in result:
            return False                     #старые прокси нам не нужны !
        select=bsObj.find("select")          #находим тег select
        opt=select.find_all("option")[12]    #смотрим что там в 12-й позиции
        if str(opt.contents[0]).strip()=="Республика Марий Эл": 
            result.append(prx)               #если то что надо, отбираем прокси
            return True                      #и сообщаем об этом
        else:
            return False                     #иначе не судьба
    except:                                  #если произошла какая-то неожиданность
        return False                         #очевидно нам этот прокси не годится
    
    
#Проверяем
pages=getHidemyProxyList(30, izbircomURL, izbircomTest)
print("Просканировано "+str(pages)+" страничек")
print("Получено "+str(len(result))+" прокси")
print(result)

На этом всё. Буду счастлив, если эта небольшая работа окажется кому-то полезной.  
Ниже очищенный код библиотеки

In [ ]:
import requests as rqs
import time
from bs4 import BeautifulSoup as soup
import threading 

def findNums(bsObj):
    res=[]                                         #результат
    script=str(bsObj.find("script").contents[0])   #находим javascript
    a=script.split("\n")                           #разбиваем по символам новой строки
    mx=0                                 #максимум числа запятых в строке
    idx=0                                #местоположение максимума
    for i in range(0, len(a)):      #ищем строку с максимальным количеством запятых
        aa=a[i].split(",")
        if len(aa)>mx:
            mx=len(aa)
            idx=i
    s=a[idx]                        #берём её часть справа от последней запятой
    s=s[s.rfind(",")+1:].strip()    #и удаляем концевые пробелы
    aa=s.split("=")                 #разбиваем по знаку =
    var=aa[0]                       #слева имя словаря
    aa=s.split('"')                 #разбиваем по знаку '"'
    var=var+"."+aa[1]               #добавляем имя поля
    aa=s.split(":")                 #разбиваем по знаку :
    res.append(aa[1][:aa[1].find("}")].strip()) #сохраняем то что справа од : до }
    aa=script.split(var)            #разбиваем весь скрипт по имени переменной
    for i in range(1, len(aa)-1):   #для всех подстрок от второй до предпоследней
        res.append(aa[i][:aa[i].find(";")]) #сохраняем от начала до ;
    return res

def evalNun(s):
    if s[0]!="+":        #если "число" начинается не с + оно не содержит 
        a=s.split("[]")  #групп в круглых скобках. Просто считаем число подстрок "[]"
        return len(a)-1
    else:
        ss=""
        a=s.split(")")              # разбиваем на группы в круглых скобках
        for i in range(0, len(a)):
            if a[i]!="":
                s=a[i][a[i].rfind("(")+1:]  #удаляем открывающиеся скобки
                aa=s.split("[]")            #разбиваем по подстрокам []
                n=0
                for q in aa:       #если длинна предшествующей строки больше 1
                    if len(q)>1:   #то это 1
                        n+=1
                ss=ss+str(n) #интерпретируем полученное число, как символ числовой строки  
        return int(ss)        
    
# Код совершенно очевиден
def evalAnsw(a):
    num=evalNun(a[0])
    for i in range(1, len(a)):
        f=False
        aa=a[i].split("=")
        nn=evalNun(aa[1])
        if aa[0]=="+":
            f=True
            num+=nn
        if aa[0]=="-":
            f=True
            num-=nn
        if aa[0]=="*":
            f=True
            num*=nn
        if f==False:
            print("Unknown "+aa[0])
    return num+len("hidemy.name")  

# Собирает до кучи все параметры
# Возвращает тройку url, cookies, param
def getParams(rsp):
    s=str(rsp.content.decode(rsp.encoding))
    bsObj=soup(s, "lxml")
    a=findNums(bsObj)
    answ=evalAnsw(a)                               #решаем пример
    form=bsObj.find("form")                        #находим форму
    url="https://hidemy.name"+form.attrs["action"] #добавляем её action к url
    inps=form.find_all("input")                    #находим все элементы input
    params={}
    for i in range(0, len(inps)):
        val=inps[i].get("value")                   #если value нет
        if val==None:                              #подставляем туда пример
            val=answ
        params[inps[i].attrs["name"]]=val           #собираем до кучи
    cookeis=rsp.cookies.get_dict()
    return (url, cookeis, params)

cookies={'__cfduid': 'dd1b26e2267ead10f15363f4066053f681517022909',
 'cf_clearance': '511ce2492b2ae5a2b2cfeb2973de73ff2921fd63-1517022914-86400'}

# Делает запрос с заданными параметрами
# Если не получает ответа 200, пытается обновить куки
# и повторить тот же запрос
def reCookies(params):
    global cookies
    firefox={
    "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8",    
    "Accept-Encoding": "gzip, deflate", 
    "Accept-Language": "ru-RU,ru;q=0.8,en-US;q=0.5,en;q=0.3",
    "Connection": "keep-alive",
    "Host": "hidemy.name",
    "Referer": "https://hidemy.name/en/proxy-list/",    
    "Upgrade-Insecure-Requests": "1",    
    "User-Agent":"Mozilla/5.0 (X11; Linux x86_64; rv:59.0) Gecko/20100101 Firefox/59.0"   
    }
    url="https://hidemy.name/en/proxy-list/"  
    if params!="":
        url=url+"?"+params
    url=url+"#list"
    for i in range(0, 10): #делаем 10 попыток
        try:
            rsp=rqs.get(url, headers=firefox, cookies=cookies)
        except:
            return None
        if rsp.status_code==200:
            return rsp
        else:
            url1, cookies, param1=getParams(rsp)  #получаем параметры обновления
            time.sleep(4)    #ждем 4 секунды и обновляемся
            try:
                rsp=rqs.get(url1, headers=firefox, params=param1, cookies=cookies)
            except:
                return None
            if rsp.status_code==200: #если получили 200, обновляем вторую куку
                rsp302=rsp.history[0]
                cookies302=rsp302.cookies.get_dict()
                cookies.update(cookies302)      
    return None #попытки закончились            
   
#https://hidemy.name/ru/proxy-list/?country=CAFRDEUS&maxtime=2000&type=h&anon=234&start=64#list
#https://hidemy.name/ru/proxy-list/?country=CAFRDEUS&maxtime=2000&ports=8080&type=h&anon=234#list
def scanHidemyPage(page, anon="4", protocol="h", speed=2000, country="", ports=""):
    res=[]  #результат
    #собираем параметры
    #Увы, стандартно в виде словаря их передавать нельзя ибо на сайте 
    #проверяется порядок их следования. Поэтому их нужно явно собирать
    #и передавать как часть строки запроса. 
    params=""
    if country!="":
        params=params+"&country="+country
    if speed!=0:
        params=params+"&maxtime="+str(speed)
    if ports!="":
        params=params+"&ports="+ports
    if protocol!="":
        params=params+"&type="+protocol
    if anon!="":
        params=params+"&anon="+anon
    if page>1:
        params=params+"&start="+str((page-1)*64)
        
    params=params[1:]    
    npages=-1              #число страниц в списке прокси
    rsp=reCookies(params)  #получаем страницу сайта
    if rsp==None:
        return (npages, res)
    try:
        s=str(rsp.content.decode(rsp.encoding))
        bsObj=soup(s, "lxml")
        #Первым делом смотрим, сколько вообще есть страничек
        pages=None
        divs=bsObj.find_all("div") #ищем все элементы div
        for div in divs:
            cc=div.attrs.get("class")
            if cc!=None:
                if cc[0]=="proxy__pagination":
                    pages=div                 #выбираем div с классом proxy__pagination
                    break
        if pages==None:
            return (-1, res)
        aa=pages.find_all("a")                # ищем все теги a
        #в тегах а ищем наибольшее число в содержимом
        for a in aa:
            ct=a.contents
            if len(ct)>0:
                nn=int(ct[0])
                if nn>npages:
                    npages=nn
        if npages<page:                        #если страница больше чем число страниц
            return (npages, res)               #читать разумеется нечего
        #читаем прокси
        trs=bsObj.find("tbody").find_all("tr") #получаем строки таблицы прокси
        #Читаем из таблицы. Каждое поле имеет свою специфику, и код не комментирую 
        for tr in trs:
            proxy={}                           #создаем новый прокси
            tds=tr.find_all("td")              #получаем td и читаем из них
            proxy["ip"]=str(tds[0].contents[0]).strip()
            proxy["port"]=str(tds[1].contents[0]).strip()
            div=tds[2].contents[0]
            proxy["country"]=str(div.contents[1]).strip()
            proxy["city"]=""
            if len(div.contents)>2:
                proxy["city"]+"|"+str(div.contents[2]).strip()
            p=tds[3].find("p")
            proxy["speed"]=int(p.contents[0].split(" ")[0])
            pt=tds[4].contents[0].split(",")
            for i in range(0, len(pt)):
                pt[i]=pt[i].strip()
            #print("speed")    
            proxy["protocol"]=pt
            anons=["No","Low", "Medium", "High"]
            an=tds[5].contents[0].strip()
            if an in anons:
                proxy["anon"]=anons.index(an)
            else:
                proxy["anon"]=-1
            ago=tds[6].contents[0].split(" ")
            proxy["ago"]=int(ago[0])*60
            if len(ago)>3:
                proxy["ago"]=60*(60*int(ago[0])+int(ago[2]))
            res.append(proxy)        
    except:
        return (npages, res)#произошла какая-то неожиданность
    return (npages, res)

#Получает список прокси с hidemy.name
#Проверяет каждый попыткой соединения c url
#проверяя результат функцией отбора selFunc
def getHidemyProxyList(nProxy, url, selFunc, hdrs={}, cooks={}, pars={}, tout=5, 
                anon="4", protocol="h", speed=2000, country="", ports=""):
    nAccept=0                    #число принятых прокси
    nth=0                        #число потоков
    lock=threading.Lock()        #замок
    done=threading.Event()       #событие окончания всех проверок

    #Вложенная функция пытающаяся соединиться проверочным сервером
    #и в случае получения какого-то ответа проверить, а тот ли это ответ что нужен
    def test(p):                
        nonlocal url, selFunc, nAccept, nth, lock, done 
        f=False
        proxy={"http":p["ip"]+":"+str(p["port"])} 
        #Соединяемся и проверяем содержимое странички
        #В случае успеха устанавливаем флаг f в True
        try: 
            rsp=rqs.get(url, proxies=proxy, cookies=cooks, params=pars, timeout=tout)
            s=str(rsp.content.decode(rsp.encoding))
            bsObj=soup(s, "lxml") 
            if selFunc(p, bsObj):
                f=True
        except:
            f=False
        lock.acquire()     #Запираем замок, тут должно быть только однопоточное исполнение
        if nth>0:          #Уменьшаем число потоков
            nth-=1
        if f:              #Если проверили успешно, увеличиваем число принятых прокси
            nAccept+=1
        if nth==0:         #Если число потоков равно нулю, запускаем событие
            done.set()
        lock.release()     #освобождаем замок
    
    page=1                 #страничка hidemy.name
    npages=10              #число страничек
    while page<=npages:    #пока есть доступные странички
        #получаем страничку
        npages, proxy=scanHidemyPage(page, anon=anon, protocol=protocol, 
                                   speed=speed, country=country, ports=ports)
        if len(proxy)==0:
            return page  #значит нечего больше делать
        for p in proxy:  #для каждого прокси запускаем проверку в отдельном потоке
            nth+=1     
            th=threading.Thread(target=test, args=[p])
            th.start()
        done.wait()      #ждем окончания проверок
        if nAccept>=nProxy:
            return page  #отваливаемся, если собрали достаточно 
        page+=1          #переходим к следующей страничке
    return page

